<a href="https://colab.research.google.com/github/mimrancomsats/Transformers/blob/main/mental_illness_bert_base_cased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 32.5 MB/s 
     |████████████████████████████████| 120 kB 68.7 MB/s 
     |████████████████████████████████| 6.6 MB 49.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 29.8 MB/s 
     |████████████████████████████████| 212 kB 58.4 MB/s 
     |████████████████████████████████| 115 kB 65.3 MB/s 
     |████████████████████████████████| 127 kB 69.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from datasets import load_dataset

In [3]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'posts_train.csv','test': 'posts_test.csv'}, encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1a0b68fcd50f47f8/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'post', 'class_name', 'class_id'],
        num_rows: 13727
    })
    test: Dataset({
        features: ['ID', 'post', 'class_name', 'class_id'],
        num_rows: 1488
    })
})

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [6]:
def transform_labels(label):

    label = label['class_name']
    num = 0
    if label == 'none':
        num = 0
    elif label == 'anxiety':
        num = 1
    elif label == 'ptsd':
        num = 2
    elif label == 'adhd':
        num = 3
    elif label == 'bipolar':
        num = 4
    elif label == 'depression':
        num = 5

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['post'], padding='max_length', truncation=True)

dataset = dataset.map(tokenize_data, batched=True)

remove_columns = ['ID', 'class_id']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/13727 [00:00<?, ?ex/s]

  0%|          | 0/1488 [00:00<?, ?ex/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['post', 'class_name', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 13727
    })
    test: Dataset({
        features: ['post', 'class_name', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1488
    })
})

In [8]:
train_dataset = dataset['train'].shuffle(seed=10).select(range(12727))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(12727, 13727))

In [9]:
train_dataset

Dataset({
    features: ['post', 'class_name', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 12727
})

In [10]:
eval_dataset

Dataset({
    features: ['post', 'class_name', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [11]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset =eval_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [12]:
#from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)

from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

1591/1591 [==============================] - 1611s 1000ms/step - loss: 1.9293 - accuracy: 0.1723 - val_loss: 2.2071 - val_accuracy: 0.1780


In [14]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [16]:
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3
1591/1591 [==============================] - 1619s 1s/step - loss: 1.0185 - accuracy: 0.6192 - val_loss: 0.6401 - val_accuracy: 0.7930
Epoch 2/3
1591/1591 [==============================] - 1598s 1s/step - loss: 0.5151 - accuracy: 0.8347 - val_loss: 0.5652 - val_accuracy: 0.8180
Epoch 3/3
1591/1591 [==============================] - 1599s 1s/step - loss: 0.2664 - accuracy: 0.9179 - val_loss: 0.5882 - val_accuracy: 0.8120
